# Install packages

In [3]:
# # Install dependencies
# # !pip3 uninstall numpy
# !pip3 install --upgrade numpy==2.0.0
# !pip3 install pandas
# !pip3 install scikit-learn mlflow seaborn shap
# !pip3 install bayesian-optimization
# !pip3 install xgboost==2.1.2
# !pip3 install optuna
# !pip3 install optuna-integration[mlflow]


# Import package

In [14]:
# Import library

import pandas as pd
import numpy as np

import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature, make_metric
from optuna.integration.mlflow import MLflowCallback
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import optuna, pickle
import os, json

# Prepare dataset
Because test dataset not have label, we must split train dataset to 2 parts. One for train and one for validate. We just do this on the first time

In [5]:
# Read dataset
# df = pd.read_csv('data/data.csv')
# train_test_data, validate_data = train_test_split(df, test_size=0.2, random_state=42)
# train_test_data.to_csv('data/train_.csv', index=False, header=True)
# validate_data.to_csv('data/validate.csv', index=False, header=True)

Now we check some information of dataset

In [6]:
# Check dataset
df = pd.read_csv('data/train_.csv')
# df.head()
# print("Dataset column")
# df.columns
# print("Summary of dataset info")
# df.info()
# print("view dimensions of dataset")
# df.shape

# for col in df.columns:
#   if df[col].dtype != 'object':  # Exclude non-numeric columns
#     min_val = df[col].min()
#     max_val = df[col].max()
#     print(f"Column: {col}")
#     print(f"Minimum: {min_val}")
#     print(f"Maximum: {max_val}")
#     print()

df.isnull().sum()


ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

We see some cell have null value, we can not drop which rows have null cell because it to much. So we just fill all null value = -1

In [7]:
def fill_nulls_with_random(df):
    def fill_column(col):
        if col.isnull().any():  # Only process columns with NaN values
            col_min = col.min()
            col_max = col.max()
            # Ensure min and max are valid for randomization
            if np.isnan(col_min) or np.isnan(col_max):
                return col  # Skip if column only contains NaNs
            col = col.apply(lambda x: np.random.uniform(col_min, col_max) if pd.isnull(x) else x)
        return col

    return df.apply(fill_column, axis=0)

# Fill NaN values with random values

random_fill = fill_nulls_with_random(df)

# data_n_null.isnull().sum()
# data_n_null.head()
# data_n_null.duplicated().sum()
# data_n_null['Label'].unique().tolist()


For tracking during training, we using MLflow. The software defined by container in mlflow folder

In [8]:
# Set mlflow as tracking server
ML_TRACKING_URL = "http://localhost:5000"
mlflow.set_tracking_uri(ML_TRACKING_URL)

# Train model
We train with some model with these steps
- We training with small part of dataset (0.2 or 0.3): dataset_frac
- We log artifacts, we see some column less contribute in  Feature Importance Score, so we delete it
- We train with full dataset, verify droped column is correct and need modify or not
- We use RandomizedSearchCV to search parameter
- We save best parameter to mlflow. With mlflow.sklearn.autolog, model and its metrics was save to model registry. We just download it and use

## Decision Tree


In [9]:
# tags = {
#     "dataset_frac": 1.0,
#     "test_size" : 0.2,
#     "droped_column" : ['ID','IPv','DNS','IRC','DHCP','ARP','SMTP','cwr_flag_number','ece_flag_number','Telnet','Drate','psh_flag_number','rst_flag_number','LLC', 'TCP','SSH','HTTPS','ack_flag_number','Std','Tot size', 'ack_count'],
#     "author": "Son Nguyen"
# }

# def dct_objective(trial):
#     with mlflow.start_run(nested=True) as run:
#         # params = {
#         #     "max_features": trial.suggest_int("max_features", 30, 100, step=2),
#         #     "criterion": trial.suggest_categorical("criterion", ["gini"]),
#         #     "max_depth": trial.suggest_int("max_depth", 1000, 1500, step=50),
#         #     "min_samples_split": trial.suggest_int("min_samples_split", 18, 30, step=2),
#         #     "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 16, step = 1),
#         #     "random_state" : 42
#         # }
#         params = {
#             "max_features": trial.suggest_int("max_features", 50, 70, step=1),
#             "criterion": trial.suggest_categorical("criterion", ["entropy"]),
#             "splitter": trial.suggest_categorical("splitter", ["best"]),
#             "max_depth": trial.suggest_int("max_depth", 1100, 1800, step=50),
#             "min_samples_split": trial.suggest_int("min_samples_split", 17, 24, step=1),
#             "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 16, step=1),
#             "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.0, 0.001, step=0.0001),
#             "ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.001, step=0.0001),
#             "class_weight": trial.suggest_categorical("class_weight", [None, "balanced"]),
#             "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 150, 250, step=5),
#             "random_state": 42
#         }
#         model = DecisionTreeClassifier(**params)

#         mlflow.sklearn.autolog()
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)
#         f1 = f1_score(y_test, y_pred, average='weighted')
#         precision = precision_score(y_test, y_pred, average='weighted')
#         recall = recall_score(y_test, y_pred, average='weighted')
#         mlflow.log_metric("accuracy", accuracy)
#         mlflow.log_metric("f1_score", f1)
#         mlflow.log_metric("precision", precision)
#         mlflow.log_metric("recall", recall)


#         feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
#         plt.figure(figsize=(20, 20))
#         sns.barplot(x=feature_scores, y=feature_scores.index)
#         plt.xlabel('Feature Importance Score')
#         plt.ylabel('Features')
#         plt.title("Visualizing Important Features")
#         feature_importance_plot = "feature_importance.png"
#         plt.savefig(feature_importance_plot, bbox_inches='tight')
#         mlflow.log_artifact(feature_importance_plot)
#         os.remove(feature_importance_plot)

        
#         metrics_file = "model_summary.json"
#         metrics = {
#             "parameter" : {**params},
#             "metrics" : {
#                 "f1" : f1,
#                 "precision" : precision,
#                 "accuracy" : accuracy,
#                 "recall" : recall
#             },
#             "droped_column" : [tags["droped_column"]]
            
#         }
        
#         with open(metrics_file, "w") as f:
#             json.dump(metrics, f, indent=4)

#         mlflow.log_artifact(metrics_file)
#         os.remove(metrics_file)

#         trial.set_user_attr("run_id", run.info.run_id)
#     return f1


# def dct_callback(study, frozen_trial):
#     winner = study.user_attrs.get("winner", None)
#     if study.best_value and winner != study.best_value:
#         study.set_user_attr("winner", study.best_value)
#         if winner:
#             improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
#             print(
#                 f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
#                 f"{improvement_percent: .4f}% improvement"
#             )
#         else:
#             print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")




# data = random_fill.drop(columns=tags['droped_column'])
# data_sample = data.sample(frac=tags['dataset_frac'])
# X = data_sample.drop(columns=['Label'])
# scaler = StandardScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# y = data_sample['Label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = 42)


# mlflow.set_experiment("decision_tree")
# optuna.logging.set_verbosity(optuna.logging.ERROR)



# with mlflow.start_run(nested=True) as run:

#     # mlflow.xgboost.autolog() can not put auto log here
#     study = optuna.create_study(direction='maximize')
#     study.optimize(dct_objective, n_trials=100, timeout=14400, callbacks=[dct_callback], show_progress_bar=True)
#     # study.optimize(dct_objective, n_trials=100, timeout=100, callbacks=[dct_callback], show_progress_bar=True)
    
#     best_trial = study.best_trial
#     best_run_id = best_trial.user_attrs['run_id']
#     # best_param = study.best_params
#     best_value = study.best_value

#     model_name = "Decision Tree"
#     client = mlflow.tracking.MlflowClient()
#     latest_ = client.get_latest_versions(model_name, stages=None)[0]


#     if latest_:
#         previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
#         if previous_f1_score >= best_value:
#             print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
#         else:
#             model_uri = f"runs:/{best_run_id}/model"
#             best_model = mlflow.register_model(model_uri, model_name)
#             best_param = client.get_run(best_run_id).data.params

#             client.update_registered_model(
#                 name=model_name,
#                 description="Best moldel",
#             )

#             for key, value in best_param.items():
#                 client.set_model_version_tag(
#                     name=model_name,
#                     version=best_model.version,
#                     key=key,
#                     value=value
#                 )

#             client.set_model_version_tag(
#                 name=model_name,
#                 version=best_model.version,
#                 key="values",
#                 value=best_value
#             )

#             for key, value in tags.items():
#                 mlflow.set_tag(key,value)
#             mlflow.set_tag("job", "Decision Tree using optuna to search parameter")


# Random Forest

In [10]:
# tags = {
#     "dataset_frac": 1.0,
#     "test_size" : 0.2,
#     # "droped_column" : ['ID'],
#     "droped_column" : ['ID','IPv', 'Telnet', 'SMTP', 'IRC', 'SSH', 'DHCP', 'ARP', 'DNS', 'Drate', 'ece_flag_number', 'cwr_flag_number', 'Duration', 'HTTP','HTTPS'],
#     "author": "Son Nguyen"
# }

# def rdf_objective(trial):
#     with mlflow.start_run(nested=True) as run:
#         params = {
#             "n_estimators": trial.suggest_int("n_estimators", 500, 1500, step=50),
#             "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
#             "criterion": trial.suggest_categorical("criterion", ["gini", "entropy","entropy"]),
#             "max_depth": trial.suggest_int("max_depth",50, 100, step=10),
#             "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 100, 500, step=50),
#             "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 0.1, step=0.01),
#             "min_weight_fraction_leaf": trial.suggest_float("min_weight_fraction_leaf", 0.01, 0.05, step=0.005),
#             "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.01, 0.05, step=0.001),
#             "ccp_alpha": trial.suggest_float("ccp_alpha", 0.01, 0.05, step=0.001),
#             "max_samples": trial.suggest_float("max_samples", 0.5, 1.0, step=0.01),
#             "class_weight": trial.suggest_categorical("class_weight", [None, "balanced", None, None]),
#             "random_state": 42,
#         }
#         # params = {
#         #     "n_estimators": trial.suggest_int("n_estimators", 50, 200, step=10),
#         #     "max_features": trial.suggest_categorical("max_features", [None, "sqrt", "log2"]),
#         #     "criterion": trial.suggest_categorical("criterion", ["log_loss", "gini", "entropy"]),
#         #     "max_depth": trial.suggest_int("max_depth", 500, 2000, step=50),
#         #     "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 100, 2000, step=50),
#         #     "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 1.0, step=0.01),
#         #     "min_weight_fraction_leaf": trial.suggest_float("min_weight_fraction_leaf", 0.01, 0.5, step=0.01),
#         #     "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.01, 1.0, step=0.01),
#         #     "ccp_alpha": trial.suggest_float("ccp_alpha", 0.01, 1.0, step=0.01),
#         #     "max_samples" : None,
#         #     "random_state": 42
#         # }
#         model = RandomForestClassifier(**params)

#         mlflow.sklearn.autolog()
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)
#         f1 = f1_score(y_test, y_pred, average='weighted')
#         precision = precision_score(y_test, y_pred, average='weighted')
#         recall = recall_score(y_test, y_pred, average='weighted')
#         mlflow.log_metric("accuracy", accuracy)
#         mlflow.log_metric("f1_score", f1)
#         mlflow.log_metric("precision", precision)
#         mlflow.log_metric("recall", recall)


#         feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
#         plt.figure(figsize=(20, 20))
#         sns.barplot(x=feature_scores, y=feature_scores.index)
#         plt.xlabel('Feature Importance Score')
#         plt.ylabel('Features')
#         plt.title("Visualizing Important Features")
#         feature_importance_plot = "feature_importance.png"
#         plt.savefig(feature_importance_plot, bbox_inches='tight')
#         mlflow.log_artifact(feature_importance_plot)
#         os.remove(feature_importance_plot)

        
#         metrics_file = "model_summary.json"
#         metrics = {
#             "parameter" : {**params},
#             "metrics" : {
#                 "f1" : f1,
#                 "precision" : precision,
#                 "accuracy" : accuracy,
#                 "recall" : recall
#             },
#             "droped_column" : [tags["droped_column"]]
            
#         }
        
#         with open(metrics_file, "w") as f:
#             json.dump(metrics, f, indent=4)

#         mlflow.log_artifact(metrics_file)
#         os.remove(metrics_file)

#         trial.set_user_attr("run_id", run.info.run_id)
#     return f1


# def rdf_callback(study, frozen_trial):
#     winner = study.user_attrs.get("winner", None)
#     if study.best_value and winner != study.best_value:
#         study.set_user_attr("winner", study.best_value)
#         if winner:
#             improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
#             print(
#                 f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
#                 f"{improvement_percent: .4f}% improvement"
#             )
#         else:
#             print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")




# data = random_fill.drop(columns=tags['droped_column'])
# data_sample = data.sample(frac=tags['dataset_frac'])
# X = data_sample.drop(columns=['Label'])
# scaler = StandardScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# y = data_sample['Label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = 42)


# mlflow.set_experiment("random-forest")
# optuna.logging.set_verbosity(optuna.logging.ERROR)



# with mlflow.start_run(nested=True) as run:

#     # mlflow.xgboost.autolog() can not put auto log here
#     study = optuna.create_study(direction='maximize')
#     study.optimize(rdf_objective, n_trials=100, timeout=14400, callbacks=[rdf_callback], show_progress_bar=True)
#     # study.optimize(dct_objective, n_trials=100, timeout=100, callbacks=[dct_callback], show_progress_bar=True)
    
#     best_trial = study.best_trial
#     best_run_id = best_trial.user_attrs['run_id']
#     # best_param = study.best_params
#     best_value = study.best_value

#     model_name = "RandomForestClassifier"
#     client = mlflow.tracking.MlflowClient()
#     latest_ = client.get_latest_versions(model_name, stages=None)[0]


#     if latest_:
#         previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
#         if previous_f1_score >= best_value:
#             print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
#         else:
#             model_uri = f"runs:/{best_run_id}/model"
#             best_model = mlflow.register_model(model_uri, model_name)
#             best_param = client.get_run(best_run_id).data.params

#             client.update_registered_model(
#                 name=model_name,
#                 description="Best moldel",
#             )

#             for key, value in best_param.items():
#                 client.set_model_version_tag(
#                     name=model_name,
#                     version=best_model.version,
#                     key=key,
#                     value=value
#                 )

#             client.set_model_version_tag(
#                 name=model_name,
#                 version=best_model.version,
#                 key="values",
#                 value=best_value
#             )

#             for key, value in tags.items():
#                 mlflow.set_tag(key,value)
#             mlflow.set_tag("job", "RandomForest Classifier using optuna to search parameter")


## XGBoost

In [13]:
tags = {
    "dataset_frac": 1.0,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID','IPv','Drate','Telnet','SMTP','ARP','cwr_flag_number','ece_flag_number','fin_flag_number','SSH','psh_flag_number','rst_flag_number','DNS', 'LLC','syn_flag_number'],
    "author": "Son Nguyen"
}

def xgboost_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "tree_method" : "hist",
            "device" : "cuda",
            "objective": "reg:squarederror",
            "n_estimators": 1000,
            "verbosity": 0,
            "eval_metric" : ["rmse", "mae", "mape", "logloss","error","auc"],
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "max_depth": trial.suggest_int("max_depth", 10, 50),
            "subsample": trial.suggest_float("subsample", 0.5, 1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.8),
            "min_child_weight": trial.suggest_int("min_child_weight", 1,15),
        }
        model = XGBClassifier(**params)
        mlflow.xgboost.autolog()
        model.fit(X_train, y_train, verbose=False)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            },
            "droped_column" : [tags["droped_column"]]
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1

def champion_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

data = random_fill.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])

mlflow.set_experiment("xgboost")
optuna.logging.set_verbosity(optuna.logging.ERROR)
with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(xgboost_objective, n_trials=100, timeout=14400, callbacks=[champion_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    best_value = study.best_value

    model_name = "XGBoost-Classifier"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]


    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
            model_uri = f"runs:/{best_run_id}/model"
            best_model = mlflow.register_model(model_uri, model_name)
            best_param = client.get_run(best_run_id).data.params
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "xgboost using optuna to search parameter")
    


  0%|          | 0/100 [00:00<?, ?it/s]

2024/12/12 13:35:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [13:35:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run bouncy-finch-672 at: http://localhost:5000/#/experiments/2/runs/a853316fa85747958a00f5bac0394ced
🧪 View experiment at: http://localhost:5000/#/experiments/2
Initial trial 0 achieved value: 0.9383339764726146


2024/12/12 14:12:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [14:12:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run enthused-robin-930 at: http://localhost:5000/#/experiments/2/runs/bf74d1883acd4ff1961efe4876a24460
🧪 View experiment at: http://localhost:5000/#/experiments/2


...

2024/12/12 17:17:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run gregarious-snipe-221 at: http://localhost:5000/#/experiments/2/runs/5fcccac25bd14701bdb7b0b6d634af20
🧪 View experiment at: http://localhost:5000/#/experiments/2


2024/12/12 17:32:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [17:32:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run spiffy-stag-516 at: http://localhost:5000/#/experiments/2/runs/989dd43c9cbd4184b80434a9beff58e1
🧪 View experiment at: http://localhost:5000/#/experiments/2
Last model is better. Current values 0.9387632372312676, latest values 0.9391637958468314
🏃 View run vaunted-pig-847 at: http://localhost:5000/#/experiments/2/runs/1c19085c59804ee4b7119b5855d6de9e
🧪 View experiment at: http://localhost:5000/#/experiments/2


C:\Users\SonNH\AppData\Local\Temp\ipykernel_21460\1426120245.py:94: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
